In [24]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [25]:
# Define the API endpoint
api_url = "http://172.16.5.6:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
data = df

In [26]:
#PARAMETER
#EWMA
alpha_ewma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [27]:
# Get mean and standard deviation of 12 periods before the last one
data['mean_12'] = data['d'].apply(lambda x: np.mean(x[-13:-1]))  # Use 12 periods before the last one
data['std_12'] = data['d'].apply(lambda x: np.std(x[-13:-1]))    # Use 12 periods before the last one

# Get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# Limit the original data to upper bound (using the 12 periods before the last one)
data['clipped_d'] = data.apply(lambda row: np.clip(row['d'][-13:-1], 0, row['ub']).tolist(), axis=1)


In [28]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))


In [38]:
import itertools
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

data['wma_clipped_d'] = data.apply(lambda row: np.clip(row['d'][-15:], 0, row['ub']).tolist(), axis=1)

def wma_forecast_with_weights(data, weights):
    wma_values = [None] * 3
    for i in range(3, len(data)):
        forecast = np.sum(np.array(data[i-3:i]) * weights) / sum(weights)
        wma_values.append(forecast)
    return wma_values

def generate_weights(step=0.05):
    weights = []
    for w1 in np.arange(0.01, 1, step):
        for w2 in np.arange(w1 + 0.01, 1 - w1, step):
            w3 = 1 - w1 - w2
            if w3 > w2 > w1 > 0 and abs(w1 + w2 + w3 - 1) < 1e-6:
                weights.append((w1, w2, w3))
    return weights

best_weights_list = []
best_maes = []

for row in data['wma_clipped_d']:
    best_mae = float('inf')
    best_weights = None
    for weights in generate_weights(step=0.05):
        wma_values = wma_forecast_with_weights(row, weights)
        mae = mean_absolute_error(row[-12:], wma_values[-12:])
        if mae < best_mae:
            best_mae = mae
            best_weights = weights
    best_weights_list.append(best_weights)
    best_maes.append(best_mae)

data['best_weights'] = best_weights_list
data['best_mae'] = best_maes

data['wma'], data['wma_result'] = zip(*data.apply(lambda row: (
    wma_forecast_with_weights(row['wma_clipped_d'], row['best_weights'])[3:][-12:],
    wma_forecast_with_weights(row['wma_clipped_d'], row['best_weights'])[-1:]
), axis=1))
print(data)


  branch agency partno                                                  d  \
0    999    99P   9901  [100, 105, 110, 108, 115, 120, 125, 130, 128, ...   
1    999    99P   9902  [200, 210, 220, 215, 230, 240, 245, 250, 255, ...   
2    999    99Q   9903  [50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...   
3    888    88A   8804  [300, 310, 320, 315, 330, 340, 345, 350, 360, ...   
4    888    88A   8805  [500, 505, 510, 515, 520, 525, 530, 535, 540, ...   
5    999    99Q   9906  [150, 155, 160, 158, 165, 170, 175, 180, 185, ...   
6    888    88B   8807  [400, 405, 410, 408, 415, 420, 425, 430, 435, ...   
7    999    99Q   9908  [80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...   
8    999    99R   9909  [250, 260, 270, 265, 280, 290, 295, 300, 310, ...   
9    888    88B   8810  [600, 610, 620, 615, 630, 640, 645, 650, 660, ...   

      mean_12     std_12          ub  \
0  130.500000  15.505375  153.758063   
1  250.833333  20.899894  282.183174   
2   80.250000  14.189932  101.53

In [30]:
# Calculate Exponential Weighted Moving Average (EWMA)
def ewma(list, alpha = alpha_ewma):
    df = pd.DataFrame(list)
    df['ewma'] = df.ewm(alpha=alpha_ewma, adjust=False).mean()
    return df['ewma'].tolist()

def ewma_forecast(list, alpha):
    ewma_values = ewma(list, alpha)
    if len(ewma_values) > 0:
        # Prediction for the next period
        next_forecast = (1 - alpha) * ewma_values[-1]
    else:
        next_forecast = None
    return ewma_values, next_forecast

data['ewma'], data['ewma_result'] = zip(*data['clipped_d'].apply(lambda x: ewma_forecast(x, alpha_ewma)))

In [31]:
#LINEAR REGRESSION
#  Calculate Linear Regression
def lr(x):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    model =  LinearRegression()
    model.fit(df[['x']], df['y'])
    df.loc[len(df), 'x'] = len(df) + 1
    return model.predict(df[['x']])

data['lr'] = data['clipped_d'].apply(lambda x: lr(x))
data['lr_result'] = data['lr'].apply(lambda x: x[-1:])


In [32]:
#POLYNOMIAL 2ND AND 3RD
# Calculate Polynomial Regression
def pr(x, pr_degree):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)

    X = df[['x']]  # Independent variable (reshape to 2D array)
    y = df['y']    # Dependent variable

    poly = PolynomialFeatures(degree=pr_degree)  # Create polynomial features
    X_poly = poly.fit_transform(X)  # Transform input features
    poly_model = LinearRegression()  # Initialize linear regression model
    poly_model.fit(X_poly, y)  # Fit polynomial model

    df.loc[len(df), 'x'] = len(df) + 1
    X_all_poly = poly.transform(df[['x']])
    return poly_model.predict(X_all_poly)  

data['pr2'] = data['clipped_d'].apply(lambda x: pr(x, 2))
data['pr2_result'] = data['pr2'].apply(lambda x: x[-1:])
data['pr3'] = data['clipped_d'].apply(lambda x: pr(x, 3))
data['pr3_result'] = data['pr3'].apply(lambda x: x[-1:])


In [33]:
#SES
def ses(x, alpha = alpha_ses):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = SimpleExpSmoothing(df['y']).fit(smoothing_level=alpha, optimized=False).fittedvalues
    return new_data.tolist()

data['ses'] = data['clipped_d'].apply(lambda x: ses(x, alpha_ses))
data['ses_result'] = data['ses'].apply(lambda x: x[-1:])

In [34]:
#DES
def des(x, alpha = alpha_ses, beta = beta_des):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = ExponentialSmoothing(df['y'], trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False).fittedvalues
    return new_data.tolist()

data['des'] = data['clipped_d'].apply(lambda x: des(x,alpha_ses, beta_des))
data['des_result'] = data['des'].apply(lambda x: x[-1:])

In [35]:
def metric(x):
    period_length = len(x['clipped_d'])
    df = pd.DataFrame()
    df['period'] = range(1, period_length + 1)
    df['qty'] = x['clipped_d'][:period_length]  # Ground truth values
    df['ma'] = x['ma'][:period_length]
    df['wma'] = x['wma'][:period_length]
    df['ewma'] = x['ewma'][:period_length]
    df['lr'] = x['lr'][:period_length]
    df['pr2'] = x['pr2'][:period_length]
    df['pr3'] = x['pr3'][:period_length]
    df['ses'] = x['ses'][:period_length]
    df['des'] = x['des'][:period_length]

    # Calculate metrics for each model
    result = []
    for model in df.columns[2:]:  # Loop through model columns (ma, ewma, etc.)
        rmse = np.sqrt(mean_squared_error(df['qty'], df[model]))  # Calculate RMSE
        r2 = r2_score(df['qty'], df[model])  # Calculate R²
        mae = mean_absolute_error(df['qty'], df[model])  # Calculate MAE
        result.append({'model': model, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    
    # Convert result to a DataFrame
    metrics_df = pd.DataFrame(result)
    
    # Select the best model (e.g., based on RMSE)
    best_model_row = metrics_df.loc[metrics_df['MAE'].idxmin()]  # Row with the lowest RMSE
    best_model = best_model_row['model']
    
    # Add the best model and metrics to the result
    return {'best_model': best_model, 'metrics': metrics_df.to_dict(orient='records')}

# Apply the metric function
data['metric'] = data.apply(lambda x: metric(x), axis=1)

# Extract the best model and metrics for each row
data['best_model'] = data['metric'].apply(lambda x: x['best_model'])
data['metrics'] = data['metric'].apply(lambda x: x['metrics'])


In [36]:
data['mean_12_FD'] = data['d'].apply(lambda x: np.mean(x[-12:]))
data['std_12_FD'] = data['d'].apply(lambda x: np.std(x[-12:]))

data['ub_FD'] = data['mean_12_FD'] + 1.5 * data['std_12_FD']

data['clipped_d_FD'] = data.apply(lambda row: np.clip(row['d'][-12:], 0, row['ub_FD']).tolist(), axis=1)
def apply_best_model_forecast(row):
    best_model = row['best_model']
    
    data = row['D'][-15:] if best_model == 'wma' else row['d'][-12:]
    
    ub = row['ub_FD']
    clipped_data = np.clip(data, 0, ub).tolist()
    
    if best_model == 'ma':
        ma_values = calculate_ma(clipped_data)
        forecast = ma_values[-1]
    elif best_model == 'ewma':
        alpha = 0.4
        weights = np.array([(1 - alpha) ** i for i in range(len(clipped_data))][::-1])
        forecast = np.sum(weights * clipped_data) / np.sum(weights)
    elif best_model == 'wma':
        weights = [0.2, 0.3, 0.5]
        if len(clipped_data) >= len(weights):
            forecast = np.sum(np.array(clipped_data[-3:]) * weights)
        else:
            forecast = np.nan
    elif best_model == 'lr':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 1)
        forecast = coef[0] * len(clipped_data) + coef[1]
    elif best_model == 'pr2':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 2)
        forecast = coef[0] * (len(clipped_data) ** 2) + coef[1] * len(clipped_data) + coef[2]
    elif best_model == 'pr3':
        X = np.arange(len(clipped_data)).reshape(-1, 1)
        y = np.array(clipped_data)
        coef = np.polyfit(X.flatten(), y, 3)
        forecast = (
            coef[0] * (len(clipped_data) ** 3)
            + coef[1] * (len(clipped_data) ** 2)
            + coef[2] * len(clipped_data)
            + coef[3]
        )
    elif best_model == 'ses':
        model = SimpleExpSmoothing(clipped_data).fit(smoothing_level=0.65, optimized=False)
        forecast = model.forecast(1)[0]
    elif best_model == 'des':
        model = Holt(clipped_data).fit(smoothing_level=0.65, smoothing_slope=0.45, optimized=False)
        forecast = model.forecast(1)[0]
    else:
        forecast = np.nan
    
    return forecast

data['FD_forecast'] = data.apply(apply_best_model_forecast, axis=1)

data['FD_final'] = data['FD_forecast'].apply(np.ceil)
data['FD_final'] = data['FD_final'].apply(lambda x: max(0, np.ceil(x)))

display(data[['partno','best_model', 'FD_forecast', 'FD_final']])
display(data)


,partno,best_model,FD_forecast,FD_final
0,9901,pr2,176.501736,177.0
1,9902,pr3,303.827646,304.0
2,9903,pr3,122.731302,123.0
3,8804,pr2,405.987970,406.0
4,8805,des,585.391622,586.0
5,9906,pr3,231.405453,232.0
6,8807,pr3,481.405453,482.0
7,9908,pr3,161.405453,162.0
8,9909,pr2,355.987970,356.0
9,8810,pr2,705.987970,706.0


,branch,agency,partno,d,mean_12,std_12,ub,clipped_d,ma,ma_result,...,des_result,metric,best_model,metrics,mean_12_FD,std_12_FD,ub_FD,clipped_d_FD,FD_forecast,FD_final
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",130.500000,15.505375,153.758063,"[110.0, 108.0, 115.0, 120.0, 125.0, 130.0, 128...","[110.0, 109.0, 111.0, 113.25, 115.6, 118.0, 11...",129.979839,...,[158.67246506190102],"{'best_model': 'pr2', 'metrics': [{'model': 'm...",pr2,"[{'model': 'ma', 'RMSE': 13.729275144473814, '...",140.666667,18.001543,167.668981,"[115.0, 120.0, 125.0, 130.0, 128.0, 135.0, 140...",176.501736,177.0
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",250.833333,20.899894,282.183174,"[220.0, 215.0, 230.0, 240.0, 245.0, 250.0, 255...","[220.0, 217.5, 221.66666666666666, 226.25, 230...",250.598598,...,[286.97208513626725],"{'best_model': 'pr3', 'metrics': [{'model': 'm...",pr3,"[{'model': 'ma', 'RMSE': 19.198605770442054, '...",263.750000,20.322914,294.234371,"[230.0, 240.0, 245.0, 250.0, 255.0, 260.0, 265...",303.827646,304.0
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",80.250000,14.189932,101.534898,"[60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88....","[60.0, 59.0, 61.0, 63.25, 65.6, 68.0, 70.42857...",79.961241,...,[103.89571713940634],"{'best_model': 'pr3', 'metrics': [{'model': 'm...",pr3,"[{'model': 'ma', 'RMSE': 12.855229698704193, '...",89.583333,15.628277,113.025749,"[65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90.0, 92....",122.731302,123.0
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",353.333333,23.033792,387.884021,"[320.0, 315.0, 330.0, 340.0, 345.0, 350.0, 360...","[320.0, 317.5, 321.6666666666667, 326.25, 330....",353.157002,...,[392.9570119901388],"{'best_model': 'pr2', 'metrics': [{'model': 'm...",pr2,"[{'model': 'ma', 'RMSE': 21.322680216645292, '...",367.083333,22.216579,400.408202,"[330.0, 340.0, 345.0, 350.0, 360.0, 365.0, 370...",405.987970,406.0
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",537.916667,17.965051,564.864243,"[510.0, 515.0, 520.0, 525.0, 530.0, 535.0, 540...","[510.0, 512.5, 515.0, 517.5, 520.0, 522.5, 525...",537.488687,...,[569.8720491107268],"{'best_model': 'des', 'metrics': [{'model': 'm...",des,"[{'model': 'ma', 'RMSE': 16.21642961963676, 'M...",549.166667,19.667373,578.667726,"[520.0, 525.0, 530.0, 535.0, 540.0, 545.0, 550...",585.391622,586.0
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",183.166667,17.591823,209.554401,"[160.0, 158.0, 165.0, 170.0, 175.0, 180.0, 185...","[160.0, 159.0, 161.0, 163.25, 165.6, 168.0, 17...",182.712867,...,[214.59341661024416],"{'best_model': 'pr3', 'metrics': [{'model': 'm...",pr3,"[{'model': 'ma', 'RMSE': 15.843080479968327, '...",194.166667,19.667373,223.667726,"[165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195...",231.405453,232.0
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",433.166667,17.591823,459.554401,"[410.0, 408.0, 415.0, 420.0, 425.0, 430.0, 435...","[410.0, 409.0, 411.0, 413.25, 415.6, 418.0, 42...",432.712867,...,[464.59341661024416],"{'best_model': 'pr3', 'metrics': [{'model': 'm...",pr3,"[{'model': 'ma', 'RMSE': 15.84308047996833, 'M...",444.166667,19.667373,473.667726,"[415.0, 420.0, 425.0, 430.0, 435.0, 440.0, 445...",481.405453,482.0
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",113.166667,17.591823,139.554401,"[90.0, 88.0, 95.0, 100.0, 105.0, 110.0, 115.0,...","[90.0, 89.0, 91.0, 93.25, 95.6, 98.0, 100.4285...",112.712867,...,[144.59341661024422],"{'best_model': 'pr3', 'metrics': [{'model': 'm...",pr3,"[{'model': 'ma', 'RMSE': 15.84308047996833, 'M...",124.166667,19.667373,153.667726,"[95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125....",161.405453,162.0
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",303.333333,23.033792,337.884021,"[270.0, 265.0, 280.0, 290.0, 295.0, 300.0, 310...","[270.0, 267.5, 271.6666666666667, 276.25, 280....",303.157002,...,[342.95701199013877],"{'best_model':